In [27]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn import svm
import pandas as pd
import glob
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
from ast import literal_eval

In [48]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, confusion_matrix, ConfusionMatrixDisplay
from sklearn import svm
from sklearn.model_selection import RandomizedSearchCV, train_test_split

In [ ]:
def string_array(x):
    for i in x:
        if i!=' '
        
    s=x.replace('    ',",")
    return s

In [ ]:
4.435275    8.196063

In [ ]:
4.435275    8.196063

In [3]:
files=glob.glob('pre_processed_training/*.csv')

In [29]:
total_training_data=pd.DataFrame([])
for f in tqdm(files):
    total_training_data=pd.concat([total_training_data,pd.read_csv(f)],axis=0)
total_training_data=total_training_data.sort_values(['Subject','Session','time_stamp'])
total_training_data["time_stamp"]=pd.to_datetime(total_training_data["time_stamp"]).round('us')
total_training_data=total_training_data[['Subject','Session','time_stamp','measurements','labels']].reset_index(drop=True)
total_training_data['measurements']=total_training_data['measurements'].apply(lambda x: np.array(literal_eval(x)))
#total_training_data=total_training_data.set_index('time_stamp').asfreq('0.025S')
#total_training_data=total_training_data.set_index(['Subject','Session','time_stamp'])

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 29/29 [00:01<00:00, 16.99it/s]


In [ ]:
import numpy as np

In [ ]:
s_time_interv= total_training_data[['Subject','Session','time_stamp']]
s_time_interv=s_time_interv.groupby(['Subject','Session']).agg({'time_stamp': [np.min,np.max]})
s_time_interv.columns=s_time_interv.columns.droplevel()
s_time_interv['time_interval']=s_time_interv['amax']-s_time_interv['amin']
s_time_interv['time_interval']=s_time_interv['time_interval'].dt.total_seconds()
s_time_interv=s_time_interv[['time_interval']]
s_time_interv=s_time_interv.unstack()
s_time_interv.columns=s_time_interv.columns.droplevel()
#s=s.reset_index(drop=True)

In [ ]:
s_time_interv.plot(kind='bar',ylabel='time_interval')

In [ ]:
s_label_dist= total_training_data[['Subject','labels','time_stamp']]
s_label_dist=s_label_dist.groupby(['Subject','labels']).count()
s_label_dist=s_label_dist.unstack()
s_label_dist.columns=s_label_dist.columns.droplevel()
s_label_dist.plot(kind='bar',xlabel='Subject',ylabel='count')

In [ ]:
s_label_dist= total_training_data[['Session','labels','time_stamp']]
s_label_dist=s_label_dist.groupby(['Session','labels']).count()
s_label_dist=s_label_dist.unstack()
s_label_dist.columns=s_label_dist.columns.droplevel()
s_label_dist.plot(kind='bar',xlabel='Session',ylabel='count')

In [ ]:
s_label_dist

In [ ]:
s_label_dist.reset_index().plot.bar()

In [ ]:
new_df = total_training_data[['time_stamp','labels','measurements']].groupby([pd.Grouper(key='time_stamp',freq='5min'),'labels']).count().reset_index().pivot(columns=['labels','time_stamp'])
#.unstack()
#new_df.columns = new_df.columns.droplevel()
#new_df=new_df.reset_index()
# new_df.plot.bar()

In [57]:
X_total=np.vstack(total_training_data.measurements)
Y_total=np.vstack(total_training_data.labels)
Y_total=Y_total.reshape((len(Y_total)))

In [58]:
X_train, X_val, y_train, y_val = train_test_split(X_total, Y_total, test_size=0.2)

In [46]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

(1341646, 1)

In [59]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
y_pred = rf.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)

RandomForestClassifier()

In [61]:
##############Predictions_Test_set###################

In [94]:
test_files=glob.glob('TestData/*.csv')

In [95]:
Subject_list= [f.split('_')[1] for f in test_files]
Session_list= [f.split('_')[2] for f in test_files]
x_sensor_dat=pd.DataFrame([])
y_sensor_dat=pd.DataFrame([])
x_time_dat=[]
for sub,sess in zip(Subject_list,Session_list):
    #x_sensor_dat=pd.DataFrame(x_sensor_dat.append(pd.read_csv(f"TrainingData/subject_{sub}_{sess}__x.csv",header=None)))
    
    x_sensor_dat=pd.concat([pd.read_csv(f"TestData/subject_{sub}_{sess}__x.csv",header=None),pd.read_csv(f"TestData/subject_{sub}_{sess}__x_time.csv",header=None).rename(columns={0:'time_stamp'})],axis=1)

    x_sensor_dat['measurements']=x_sensor_dat.apply(lambda x: [x[0],x[1],x[2],x[3],x[4],x[5]],axis=1)
    x_sensor_dat=x_sensor_dat[['time_stamp','measurements']]
    x_sensor_dat["time_stamp"]=pd.to_datetime(x_sensor_dat["time_stamp"],unit='s').round('us')
    #x_sensor_dat=x_sensor_dat.set_index('time_stamp').asfreq('0.025S')
    x_sensor_dat['Subject']=int(sub)
    x_sensor_dat['Session']=int(sess)
    #x_sensor_dat=x_sensor_dat.set_index('time_stamp').asfreq('0.025S')

In [99]:
x_sensor_dat.shape

(51761, 4)

In [102]:
X_test=np.vstack(x_sensor_dat.measurements)
#Y_total=np.vstack(total_training_data.labels)
#Y_total=Y_total.reshape((len(Y_total)))

In [103]:
X_test.shape

(51761, 6)

In [104]:
test_pred = rf.predict(X_test)
#accuracy = accuracy_score(y_val, y_pred)

In [106]:
test_pred.shape

(51761,)

In [109]:
x_sensor_dat['test_pred']=list(test_pred)

In [110]:
x_sensor_dat

,time_stamp,measurements,Subject,Session,test_pred
0,1970-01-01 00:00:00.000,"[0.4845667, 8.438189, 4.87, -0.004444444, -0.0...",11,1,0
1,1970-01-01 00:00:00.025,"[0.4607261, 8.44629, 4.92, -0.002238946, -0.00...",11,1,0
2,1970-01-01 00:00:00.050,"[0.4782264, 8.45, 4.889409, -0.004970017, -0.0...",11,1,0
3,1970-01-01 00:00:00.075,"[0.4600064, 8.429994, 4.890019, 0.002450463, 0...",11,1,0
4,1970-01-01 00:00:00.100,"[0.4786646, 8.438665, 4.894006, 0.003333333, 0...",11,1,0
...,...,...,...,...,...
51756,1970-01-01 00:21:33.900,"[-0.2991281, 9.78, 0.164564, -0.001450906, -0....",11,1,0
51757,1970-01-01 00:21:33.925,"[-0.3144154, 9.777923, 0.1944154, -0.002222222...",11,1,0
51758,1970-01-01 00:21:33.950,"[-0.3067322, 9.786634, 0.1833661, 0.00186618, ...",11,1,0
51759,1970-01-01 00:21:33.975,"[-0.2998966, 9.769897, 0.1949483, 0.0003990475...",11,1,0


In [111]:
x_sensor_dat.groupby('test_pred')[['Session']].count()

,Session
test_pred,
0,47281
1,886
2,2034
3,1560
